In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.decomposition import PCA
from pprint import pprint
from sklearn.metrics import f1_score,r2_score,classification_report,roc_auc_score
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,RandomForestRegressor
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
import itertools
from sklearn.model_selection import ParameterGrid, cross_val_predict, GroupKFold,GridSearchCV,StratifiedKFold
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from joblib import Parallel,delayed

def svar(X):
    n = float(len(X))
    svar=(sum([(x-np.mean(X))**2 for x in X]) / n)* n/(n-1.)
    return svar

def CronbachAlpha(itemscores):
    itemvars = [svar(item) for item in itemscores]
    tscores = [0] * len(itemscores[0])
    for item in itemscores:
        for i in range(len(item)):
            tscores[i]+= item[i]
    nitems = len(itemscores)
    Calpha=nitems/(nitems-1.) * (1-sum(itemvars)/ svar(tscores))
    return Calpha

from scipy.stats import iqr,skew,kurtosis,pearsonr
from scipy.stats import variation,moment

def get_features1(x):
    temp = []
    temp.append(x[-1])
    temp.append(x[0])
    temp.append(x[len(x)//2])
    temp.append(iqr(x))
    temp.append(skew(x))
    temp.append(kurtosis(x))
    temp.append(variation(x))
    temp.append(iqr(x[:len(x)//2]))
    temp.append(skew(x[:len(x)//2]))
    temp.append(kurtosis(x[:len(x)//2]))
    temp.append(variation(x[:len(x)//2]))
    temp.append(iqr(x[len(x)//2:]))
    temp.append(skew(x[len(x)//2:]))
    temp.append(kurtosis(x[len(x)//2:]))
    temp.append(variation(x[len(x)//2:]))
    return np.array(temp)

# def get_features()

def get_features(temp,user):
    if temp.shape[0]<5:
        return -1,-1,-1,-1,-1
    if temp['time'].values[-1]-temp['time'].values[0]>1000:
        return -1,-1,-1,-1,-1
#     if np.isnan(temp['score'].values[-1]):
#         return -1,-1,-1,-1,-1
    a = temp['stress_likelihood'].values
    b = temp['all_scores'].values[-1]
    c = user
    hour = datetime.fromtimestamp(temp['time'].values[0]).hour
    if hour<8:
        d = 0
    elif hour>8 and hour < 16:
        d = 1
    else:
        d = 2
    e = user+temp['day'].values[-1]
    return a,b,c,d,e

def get_user_data(user_data,user):
    X_date = []
    X = []
    y = []
    groups = []
    days = []
    if len(np.unique(user_data.dropna()['day'].values))<4:
        return [],[],[],[],[]
    user_data = user_data.sort_values('localtime').reset_index(drop=True)
    count = 0
    columns=['window', 'localtime', 'day', 'stress_likelihood', 'imputed',
       'likelihood_mean', 'all_scores', 'score', 'label', 'version', 'user',
       'time']
    count = 0
    data = Parallel(n_jobs=20,verbose=1)(delayed(get_features)(user_data.loc[i:i+10],user) for i,row in user_data.iterrows()
                                        if not np.isnan(user_data.loc[i:i+10]['score'].values[-1]))
    for a in data:
        if not isinstance(a[2], str):
            continue
        X.append(a[0])
        y.append(a[1])
        groups.append(a[2])
        X_date.append(a[3])
        days.append(a[4])
    return X,y,groups,X_date,days


def get_data(all_stress):
    X_date,X,y,groups,days = [],[],[],[],[]
    final = [get_user_data(all_stress[all_stress.user.isin([user])],user) 
                                          for user in np.unique(all_stress['user'].values)]
    for a in final:
        X.extend(a[0])
        y.extend(a[1])
        groups.extend(a[2])
        X_date.extend(a[3])
        days.extend(a[4])
    return X,y,groups,X_date,days

In [ ]:
import warnings
import pickle
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler,OneHotEncoder
all_stress_left = pickle.load(open('../data/stress_ema_md2k_aa_rice_left_ppg.p','rb'))
# all_stress_left = all_stress_left[all_stress_left.imputed.isin([0])]
all_stress_right = pickle.load(open('../data/stress_ema_md2k_aa_rice_right1_ppg.p','rb'))
# all_stress_right = all_stress_right[all_stress_right.imputed.isin([0])]

In [ ]:
X2,y2,groups2,X_date2,days2 = get_data(all_stress_left)
X3,y3,groups3,X_date3,days3 = get_data(all_stress_right)
Xf,yf,groupsf,X_datef,daysf = X2+X3,y2+y3,groups2+groups3,X_date2+X_date3,days2+days3

In [ ]:
pickle.dump([Xf,yf,groupsf,X_datef,daysf],open('../data/ema_only_imputed.p','wb'))

In [ ]:
from sklearn import linear_model
Xf,yf,groupsf,X_datef,daysf = pickle.load(open('../data/ema_only_imputed.p','rb'))
X,y1,groups1,X_date_final = np.array(Xf),np.array(yf),np.array(groupsf),OneHotEncoder().fit_transform(np.array(X_datef).reshape(-1,1))
Xfeatures = np.array([get_features1(x) for x in X])
all_y = []
all_y_pred = []
all_groups = []
for user in np.unique(groups1):
    index = np.where(groups1==user)[0]
    X1,y,groups = Xfeatures[index],y1[index],groups1[index]
    labels = []
    temp = np.array([np.mean(a) for a in y])
    temp_mean = np.mean(temp)
    index2 = np.where(temp>temp_mean)[0]
    index1 = np.where(temp<=temp_mean)[0]
    temp[index2] = 1
    temp[index1] = 0
    labels = temp
    labels = np.int64(np.array(labels))
    if len(np.unique(labels))==1:
        continue
    if len(labels[labels==1])<3:
        continue
    if len(labels)<10:
        continue
    paramGrid = {
#             'rf__kernel':['linear'],
            'rf__C': np.logspace(-3,3,100),
#             'rf__gamma': np.logspace(-6,3,2),
            'rf__class_weight':[{0:1,1:4/2},{0:1,1:3/2},{0:1,1:5/2},{0:1,1:1}],
            'pca__n_components':[3,4]
        }
    y = labels
    clf = Pipeline([('pca',PCA()),('rf',linear_model.LogisticRegression())])
    grid_search = GridSearchCV(clf, paramGrid, n_jobs=-1,cv=StratifiedKFold(n_splits=3).split(X1, y),
                               scoring='f1',verbose=1, return_train_score=False)
    grid_search.fit(X1,y)
    from sklearn.metrics import classification_report
    clf = grid_search.best_estimator_
    y_pred = cross_val_predict(clf,X1,y,cv=StratifiedKFold(n_splits=3).split(X1, y))
    all_y.extend(list(y))
    all_y_pred.extend(list(y_pred))
    all_groups.extend(list(groups))
    print(confusion_matrix(y,y_pred),classification_report(y,y_pred))

In [ ]:
print(confusion_matrix(all_y,all_y_pred),classification_report(all_y,all_y_pred))

In [ ]:
getf1(Xfeatures,labels,groups)
# y.shape

In [ ]:
from frameworks.CPLELearning

In [ ]:
X[0]